# Create Spectral Library

## Introduction

This notebook extracts average vegetation reflectance spectra from a collection of airborne hyperspectral image files and a table of field points (latitude, longitude and metadata). For each point, we look up which image scene covers that location, read the pixel around the point, filter out non-vegetation pixels using Normalised Difference Vegetation Index (NDVI), average the spectra of the pixels (If the point lies between two or more pixels), and save the results as a local spectral library **.csv** and a wavelengths **.csv**.


The purpose of this notebook is to provide a workflow of building a simple spectral library for vegetation using hyperspectral images (in a format readable by rasterio) and a csv of point locations. 


## Inputs required

- A list of hyperspectral image scene files (images should be accessible at the paths in the `scene_list`). The notebook will check for common extensions (**.img**, **.dat**, **.bin**)
- A **.csv** file with columns named exactly `Latitude` and `Longitude` (plus any metadata columns you want copied into the library output).

## Outputs produced

- A **.csv** file containing the extracted spectra with metadata columns, the scene name, and the number of pixels used for the averaged spectrum.
- A **.csv** file containing the wavelengths associated with the kept bands (band index and wavelength in nanometres).


Before running the code, make sure the paths in the `scene_list` point to your data on disk and that `points_csv` leads to your **.csv** file. If your image files have a different extension, the notebook will test common ones and fail if it cannot find the files.

### Import python modules

The code below imports the python libraries we will use. if you do not have a library installed (for example rasterio or geopandas) install it first using pip or conda.

In [3]:
import os
import re
import math
import numpy as np
import pandas as pd
import rasterio as rio
from shapely.geometry import box
import geopandas as gpd

### Load data and configure

The cell below contains:

- `scene_list`: a python list of paths for each hyperspectral scene you want to use.
- `points_csv`: path to the csv file that contains `Latitude` and `Longitude` columns.
- `out_library_csv` and `out_wavelengths_csv`: paths for outputs produced by this notebook.
- Window and filtering parameters: radius around each point to read, the minimum NDVI threshold to consider a pixel 'vegetation', and tile step for faster reading (not used in this script but left as a parameter).


**_Change the paths and parameters below to match your data and desired behavior._**

In [4]:
scene_list = [
    r"data/AVNG_L2/ang20231109t125547_021_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_000_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_001_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_002_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_003_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_004_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_005_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_006_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_007_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_008_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_009_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t130728_013_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_008_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_009_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_010_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_011_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_012_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_013_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_014_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_016_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_017_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_018_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_019_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_020_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t131923_021_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_000_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_001_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_002_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_003_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_004_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_005_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_006_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_007_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_008_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_009_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_010_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_011_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_012_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_013_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t133124_014_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t134249_007_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t134249_008_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t134249_009_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t134249_010_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_003_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_004_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_005_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_006_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_008_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_009_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_010_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t135449_011_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t140547_015_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t140547_016_L2A_OE_main_27577724_RFL_ORT",
    r"data/AVNG_L2/ang20231109t141759_010_L2A_OE_main_27577724_RFL_ORT"
]                                                                           # Almost all the scenes around Table Mountain Nature Reserve
points_csv          = r"final/ReDo/Vegetation_Library.csv"
out_library_csv     = r"final/ReDo/local_spectral_library_aviris.csv"
out_wavelengths_csv = r"final/ReDo/local_spectral_library_wavelengths.csv"

# Extraction window and filters:
window_radius = 0        # Radius in pixels; 1 -> 3x3 window, 2 -> 5x5 window around the location
ndvi_minimum = 0.30     # Keep only vegetation-like pixels with ndvi greater than this
tile_step = 512         # Tile for speed (not used directly here but kept as configuration)

# Returns a boolean mask for wavelengths to keep based on simple bad-band rules.
# This removes strong atmospheric absorption regions and extremes outside sensor range.
def keep_mask_nm(wavelength_nm):
    return ~((wavelength_nm < 450) |
             ((wavelength_nm >= 1340) & (wavelength_nm <= 1480)) |
             ((wavelength_nm >= 1800) & (wavelength_nm <= 1980)) |
             (wavelength_nm > 2400))

### Resolve scene file path

The code below checks each path in `scene_list` and tries common file extensions (**.img**, **.dat**, **.bin**).
If a file is not found the code will stop with an error so you can correct the path.

In [5]:
resolved_scene_list = []
for scene_stem in scene_list:
    for extension in ("", ".img", ".dat", ".bin"):
        candidate_path = scene_stem + extension
        if os.path.exists(candidate_path):
            resolved_scene_list.append(candidate_path)
            break
    else:
        raise SystemExit(f"cannot find data file for scene stem: {scene_stem}")

scene_list = resolved_scene_list  # replace original list with resolved full paths

### Read wavelengths from the first scene .hdr and decide which spectral bands to keep

We read the **.hdr** file (a text header produced by many hyperspectral formats) and extract the wavelength list. Then we build a boolean mask to remove bad bands (atmospheric absorption regions and extremes). We also choose two wavelengths to compute NDVI (red and near-infrared band indices) from the kept wavelengths.

In [6]:
hdr_filepath = scene_list[0] + ".hdr"
hdr_text = open(hdr_filepath, "r", errors="ignore").read()
m = re.search(r"wavelength\s*=\s*{([^}]*)}", hdr_text, flags=re.I|re.S)
if not m:
    raise SystemExit("no 'wavelength' block found in the .hdr file")
wavelengths_all = np.array([float(x) for x in re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", m.group(1))], float)
keep_indices = np.where(keep_mask_nm(wavelengths_all))[0]
wavelengths_kept = wavelengths_all[keep_indices]
print(f"kept bands: {keep_indices.size} of {wavelengths_all.size}")

# Pick NDVI bands among kept wavelengths (closest to ≈665 nm and ≈810 nm)
ndvi_red_index = np.argmin(np.abs(wavelengths_kept - 665.0))
ndvi_nir_index = np.argmin(np.abs(wavelengths_kept - 810.0))

Kept bands: 325 of 425


### Load points and set CRS

The **.csv** must have `Latitude` and `Longitude` columns. we convert the dataframe into a geopandas geodataframe with wgs84 (epsg:4326) so we can reproject it to each scene's CRS when extracting pixels.

In [7]:
points_df = pd.read_csv(points_csv)
if not {"Latitude", "Longitude"}.issubset(points_df.columns):
    raise SystemExit("csv must have 'Latitude' and 'Longitude' columns")
points_gdf = gpd.GeoDataFrame(points_df.copy(),
                              geometry=gpd.points_from_xy(points_df.Longitude, points_df.Latitude),
                              crs=4326)

### Extract Spectra

For each scene in the scene list:
1. Open the scene with `rasterio` and get its crs and bounds.
2. Reproject the points to the scene crs and keep only the points that intersect the scene bounding box.
3. For each point, compute the pixel row/column. Read the small window of pixels around the point (depending on `window_radius`).
4. Read only the wavelengths/bands we decided to keep from the header (kept indices). The data returned is shaped (kept_bands, h, w).
5. Flatten the window to a list of pixel spectra and remove masked/invalid pixels.
6. Compute NDVI for each pixel and keep only pixels with NDVI > ndvi_minimum.
7. Average the spectra of the remaining vegetation pixels and record metadata, scene name and number of pixels used.
8. Append this row to the library list.

This process produces a list of spectra ready to be saved to **.csv**.

In [8]:
rows = []  # list of dictionaries; each dictionary becomes a row in the output csv
for scene_index, scene_path in enumerate(scene_list, start=1):
    with rio.open(scene_path) as src:
        scene_crs = src.crs
        scene_height, scene_width = src.height, src.width
        xmin, ymin, xmax, ymax = src.bounds
        # Reproject points to the scene crs and keep only those inside the scene bounds
        points_in_scene = points_gdf.to_crs(scene_crs)
        points_in_scene = points_in_scene.loc[points_in_scene.geometry.intersects(box(xmin, ymin, xmax, ymax))]
        print(f"[scene {scene_index}] {os.path.basename(scene_path)}  candidates: {len(points_in_scene)}")
        if points_in_scene.empty:
            continue
        for _, point_row in points_in_scene.iterrows():
            # compute the row and column for the geographic location
            row, col = src.index(point_row.geometry.x, point_row.geometry.y)
            r0 = max(0, row - window_radius)
            r1 = min(scene_height, row + window_radius + 1)
            c0 = max(0, col - window_radius)
            c1 = min(scene_width, col + window_radius + 1)
            if r1 <= r0 or c1 <= c0:
                continue
            # Read only the kept bands from the image within the window
            cube = src.read(indexes=(keep_indices + 1).tolist(),
                            window=rio.windows.Window(c0, r0, c1 - c0, r1 - r0),
                            masked=True)  # shape: (kept_bands, h, w)
            if cube.size == 0:
                continue
            # Reshape to (n_pixels, n_bands)
            pixel_array = cube.reshape(keep_indices.size, -1).T
            # Handle masked arrays (masked pixels) or invalid values
            if hasattr(pixel_array, "mask"):
                valid_mask = ~np.any(pixel_array.mask, axis=1)
                pixel_array = np.array(pixel_array)
            else:
                valid_mask = np.isfinite(pixel_array).all(1)
            valid_pixels = pixel_array[valid_mask]
            if valid_pixels.size == 0:
                continue
            # Compute NDVI for each pixel and keep only vegetation-like pixels
            ndvi_values = (valid_pixels[:, ndvi_nir_index] - valid_pixels[:, ndvi_red_index]) / (valid_pixels[:, ndvi_nir_index] + valid_pixels[:, ndvi_red_index] + 1e-12)
            vegetation_pixels = valid_pixels[ndvi_values > ndvi_minimum]
            if vegetation_pixels.size == 0:
                continue
            # Compute the mean spectrum over the vegetation pixels
            mean_spectrum = vegetation_pixels.mean(0)
            # Copy metadata columns from the original csv (not including geometry)
            metadata = {k: point_row[k] for k in points_df.columns if k not in ("geometry",)}
            out_row = {**metadata, "scene": os.path.basename(scene_path), "number_of_pixels": int(vegetation_pixels.shape[0])}
            # Add spectral bands as b1, b2, ... using the kept band ordering
            for band_index, band_value in enumerate(mean_spectrum, start=1):
                out_row[f"b{band_index}"] = float(band_value)
            rows.append(out_row)

[scene 1] ang20231109t125547_021_L2A_OE_main_27577724_RFL_ORT  candidates: 4
[scene 2] ang20231109t130728_000_L2A_OE_main_27577724_RFL_ORT  candidates: 3
[scene 3] ang20231109t130728_001_L2A_OE_main_27577724_RFL_ORT  candidates: 5
[scene 4] ang20231109t130728_002_L2A_OE_main_27577724_RFL_ORT  candidates: 9
[scene 5] ang20231109t130728_003_L2A_OE_main_27577724_RFL_ORT  candidates: 16
[scene 6] ang20231109t130728_004_L2A_OE_main_27577724_RFL_ORT  candidates: 3
[scene 7] ang20231109t130728_005_L2A_OE_main_27577724_RFL_ORT  candidates: 4
[scene 8] ang20231109t130728_006_L2A_OE_main_27577724_RFL_ORT  candidates: 0
[scene 9] ang20231109t130728_007_L2A_OE_main_27577724_RFL_ORT  candidates: 0
[scene 10] ang20231109t130728_008_L2A_OE_main_27577724_RFL_ORT  candidates: 29
[scene 11] ang20231109t130728_009_L2A_OE_main_27577724_RFL_ORT  candidates: 14
[scene 12] ang20231109t130728_013_L2A_OE_main_27577724_RFL_ORT  candidates: 25
[scene 13] ang20231109t131923_008_L2A_OE_main_27577724_RFL_ORT  candi

### Write outputs

This cell converts the list of extracted spectra into a pandas dataframe and writes two **.csv** files:
- The spectral library **.csv** (`out_library_csv`) has one row per extracted mean-spectrum and contains the original metadata columns plus `scene` and `number_of_pixels` and columns `b1..bN` for each kept band.
- The wavelengths **.csv** (`out_wavelengths_csv`) contains two columns: `band` (index in the kept-band ordering) and `wavelength_nm` (wavelength in nanometres).

If no spectra were extracted the code raises an error so you can verify NDVI threshold, points and scene coverage.

In [9]:
if not rows:
    raise SystemExit("no spectra extracted — check points vs scenes, crs and ndvi threshold.")
library_df = pd.DataFrame(rows)
os.makedirs(os.path.dirname(out_library_csv), exist_ok=True)
library_df.to_csv(out_library_csv, index=False)
pd.DataFrame({"band": np.arange(1, keep_indices.size + 1), "wavelength_nm": wavelengths_kept}).to_csv(out_wavelengths_csv, index=False)
print(f"wrote library: {out_library_csv}  ({library_df.shape[0]} spectra × {keep_indices.size} bands)")
print(f"wrote wavelengths: {out_wavelengths_csv}")

Wrote library: final/ReDo/local_spectral_library_aviris.csv  (486 spectra × 325 bands)
Wrote wavelengths: final/ReDo/local_spectral_library_wavelengths.csv
